In [14]:
import sys
from scipy.io import wavfile
from numpy import mean, absolute, array_equal
from waveform_analysis import A_weight, ITU_R_468_weight
from waveform_analysis._common import rms_flat, dB
import matplotlib.pyplot as plt


try:
    from soundfile import SoundFile
    wav_loader = 'pysoundfile'
except:
    try:
        from scikits.audiolab import Sndfile
        wav_loader = 'scikits.audiolab'
    except:
        try:
            from scipy.io.wavfile import read
            wav_loader = 'scipy.io.wavfile'
        except:
            raise ImportError('No sound file loading package installed '
                              '(PySoundFile, scikits.audiolab, or SciPy)')

try:
    import easygui
    displayer = 'easygui'
except:
    displayer = 'stdout'


def display(header, results):
    """
    Display header string and list of result lines
    """
    if displayer == 'easygui':
        title = 'Waveform properties'
        easygui.codebox(header, title, '\n'.join(results))
    else:
        #print('No EasyGUI; printing output to console\n')
        print(header)
        print('-----------------')
        print('\n'.join(results))


def histogram(signal):
    """
    Plot a histogram of the sample values
    """
    try:
        from matplotlib.pyplot import hist, show
    except ImportError:
        print('Matplotlib not installed - skipping histogram')
    else:
        print('Plotting histogram')
        hist(signal)  # TODO: parameters, abs(signal)?
        show()


def properties(signal, sample_rate):
    """
    Return a list of some wave properties for a given 1-D signal
    """
    # Measurements that include DC component
    DC_offset = mean(signal)
    # Maximum/minimum sample value
    # Estimate of true bit rate

    # Remove DC component
    signal -= mean(signal)

    # Measurements that don't include DC
    signal_level = rms_flat(signal)
    peak_level = max(absolute(signal))
    crest_factor = peak_level/signal_level

    # Apply the A-weighting filter to the signal
    Aweighted = A_weight(signal, sample_rate)
    Aweighted_level = rms_flat(Aweighted)

    # Apply the ITU-R 468 weighting filter to the signal
    ITUweighted = ITU_R_468_weight(signal, sample_rate)
    ITUweighted_level = rms_flat(ITUweighted)

    # TODO: rjust instead of tabs

    return [
        'DC offset:\t%f (%.3f%%)' % (DC_offset, DC_offset * 100),
        'Crest factor:\t%.3f (%.3f dB)' % (crest_factor, dB(crest_factor)),
        'Peak level:\t%.3f (%.3f dBFS)' %
        (peak_level, dB(peak_level)),  # Doesn't account for intersample peaks!
        'RMS level:\t%.3f (%.3f dBFS)' % (signal_level, dB(signal_level)),
        'RMS A-weighted:\t%.3f (%.3f dBFS(A), %.3f dB)' %
        (Aweighted_level, dB(Aweighted_level),
         dB(Aweighted_level/signal_level)),
        'RMS 468-weighted:\t%.3f (%.3f dBFS(468), %.3f dB)' %
        (ITUweighted_level, dB(ITUweighted_level),
         dB(ITUweighted_level/signal_level)),
        '-----------------',
    ]


def analyze(filename):
    if wav_loader == 'pysoundfile':
        sf = SoundFile(filename)
        signal = sf.read()
        channels = sf.channels
        sample_rate = sf.samplerate
        samples = len(sf)
        file_format = sf.format_info + ' ' + sf.subtype_info
        sf.close()
    elif wav_loader == 'scikits.audiolab':
        sf = Sndfile(filename, 'r')
        signal = sf.read_frames(sf.nframes)
        channels = sf.channels
        sample_rate = sf.samplerate
        samples = sf.nframes
        file_format = sf.format
        sf.close()
    elif wav_loader == 'scipy.io.wavfile':
        sample_rate, signal = read(filename)
        try:
            channels = signal.shape[1]
        except IndexError:
            channels = 1
        samples = signal.shape[0]
        file_format = str(signal.dtype)

        # Scale common formats
        # Other bit depths (24, 20) are not handled by SciPy correctly.
        if file_format == 'int16':
            signal = signal.astype(float) / (2**15)
        elif file_format == 'uint8':
            signal = (signal.astype(float) - 128) / (2**7)
        elif file_format == 'int32':
            signal = signal.astype(float) / (2**31)
        elif file_format == 'float32':
            pass
        else:
            raise Exception("Don't know how to handle file "
                            "format {}".format(file_format))

    else:
        raise Exception("wav_loader has failed")

    header = '\n\n\ndBFS values are relative to a full-scale square wave'

    if samples/sample_rate >= 1:
        length = str(samples/sample_rate) + ' seconds'
    else:
        length = str(samples/sample_rate*1000) + ' milliseconds'

    results = [
        "Using sound file backend '" + wav_loader + "'",
        'Properties for "' + filename + '"',
        str(file_format),
        'Channels:\t%d' % channels,
        'Sampling rate:\t%d Hz' % sample_rate,
        'Samples:\t%d' % samples,
        'Length: \t' + length,
        '-----------------',
        ]

    if channels == 1:
        # Monaural
        results += properties(signal, sample_rate)
    elif channels == 2:
        # Stereo
        if array_equal(signal[:, 0], signal[:, 1]):
            results += ['Left and Right channels are identical:']
            results += properties(signal[:, 0], sample_rate)
        else:
            results += ['Left channel:']
            results += properties(signal[:, 0], sample_rate)
            results += ['Right channel:']
            results += properties(signal[:, 1], sample_rate)
    else:
        # Multi-channel
        for ch_no, channel in enumerate(signal.transpose()):
            results += ['Channel %d:' % (ch_no + 1)]
            results += properties(channel, sample_rate)

    display(header, results)
    
    freq_from_fft(signal, sample_rate)
    THD(signal, sample_rate)
    #THDN(signal, sample_rate)
         
    plot_histogram = False
    if plot_histogram:
        histogram(signal)


def wave_analyzer(files):
    if files:
        for filename in files:
            try:
                analyze(filename)
            except IOError:
                print('Couldn\'t analyze "' + filename + '"\n')
            print('')
    else:
        # TODO: realtime analyzer goes here
        sys.exit("You must provide at least one file to analyze:\n"
                 "python wave_analyzer.py filename.wav")

#!/usr/bin/env python

from waveform_analysis._common import parabolic
from numpy.fft import rfft
from numpy import asarray, argmax, mean, diff, log, copy
from waveform_analysis._common import find
from scipy.signal import correlate, kaiser, decimate


def freq_from_crossings(signal, fs, interp='linear'):
    """
    Estimate frequency by counting zero crossings

    Works well for long low-noise sines, square, triangle, etc.

    Pros: Fast, accurate (increasing with signal length).

    Cons: Doesn't work if there are multiple zero crossings per cycle,
    low-frequency baseline shift, noise, inharmonicity, etc.
    """
    signal = asarray(signal) + 0.0

    # Find all indices right before a rising-edge zero crossing
    indices = find((signal[1:] >= 0) & (signal[:-1] < 0))

    if interp == 'linear':
        # More accurate, using linear interpolation to find intersample
        # zero-crossings (Measures 1000.000129 Hz for 1000 Hz, for instance)
        crossings = [i - signal[i] / (signal[i+1] - signal[i])
                     for i in indices]
    elif interp == 'none' or interp is None:
        # Naive (Measures 1000.185 Hz for 1000 Hz, for instance)
        crossings = indices
    else:
        raise ValueError('Interpolation method not understood')

        # TODO: Some other interpolation based on neighboring points might be
        # better.  Spline, cubic, whatever  Can pass it a function?

    return fs / mean(diff(crossings))


def freq_from_fft(signal, fs):
    """
    Estimate frequency from peak of FFT

    Pros: Accurate, usually even more so than zero crossing counter
    (1000.000004 Hz for 1000 Hz, for instance).  Due to parabolic
    interpolation being a very good fit for windowed log FFT peaks?
    https://ccrma.stanford.edu/~jos/sasp/Quadratic_Interpolation_Spectral_Peaks.html
    Accuracy also increases with signal length

    Cons: Doesn't find the right value if harmonics are stronger than
    fundamental, which is common.
    """
    signal = asarray(signal)

    N = len(signal)

    # Compute Fourier transform of windowed signal
    windowed = signal * kaiser(N, 100)
    f = rfft(windowed)

    # Find the peak and interpolate to get a more accurate peak
    i_peak = argmax(abs(f))  # Just use this value for less-accurate result
    i_interp = parabolic(log(abs(f)), i_peak)[0]

    # Convert to equivalent frequency
    print('Frequency:%f' % (fs * i_interp / N))  # Hz


def freq_from_autocorr(signal, fs):
    """
    Estimate frequency using autocorrelation

    Pros: Best method for finding the true fundamental of any repeating wave,
    even with strong harmonics or completely missing fundamental

    Cons: Not as accurate, doesn't find fundamental for inharmonic things like
    musical instruments, this implementation has trouble with finding the true
    peak
    """
    signal = asarray(signal) + 0.0

    # Calculate autocorrelation, and throw away the negative lags
    signal -= mean(signal)  # Remove DC offset
    corr = correlate(signal, signal, mode='full')
    corr = corr[len(corr)//2:]

    # Find the first valley in the autocorrelation
    d = diff(corr)
    start = find(d > 0)[0]

    # Find the next peak after the low point (other than 0 lag).  This bit is
    # not reliable for long signals, due to the desired peak occurring between
    # samples, and other peaks appearing higher.
    i_peak = argmax(corr[start:]) + start
    i_interp = parabolic(corr, i_peak)[0]

    return fs / i_interp


def freq_from_hps(signal, fs):
    """
    Estimate frequency using harmonic product spectrum

    Low frequency noise piles up and overwhelms the desired peaks

    Doesn't work well if signal doesn't have harmonics
    """
    signal = asarray(signal) + 0.0

    N = len(signal)
    signal -= mean(signal)  # Remove DC offset

    # Compute Fourier transform of windowed signal
    windowed = signal * kaiser(N, 100)

    # Get spectrum
    X = log(abs(rfft(windowed)))

    # Remove mean of spectrum (so sum is not increasingly offset
    # only in overlap region)
    X -= mean(X)

    # Downsample sum logs of spectra instead of multiplying
    hps = copy(X)
    for h in range(2, 9):  # TODO: choose a smarter upper limit
        dec = decimate(X, h, zero_phase=True)
        hps[:len(dec)] += dec

    # Find the peak and interpolate to get a more accurate peak
    i_peak = argmax(hps[:len(dec)])
    i_interp = parabolic(hps, i_peak)[0]

    # Convert to equivalent frequency
    return fs * i_interp / N  # Hz

from scipy.signal.windows import general_cosine
from scipy.fftpack import next_fast_len
from numpy.fft import rfft, irfft
from numpy import argmax, mean, log, concatenate, zeros
import numpy as np
from waveform_analysis._common import rms_flat, parabolic
from waveform_analysis import A_weight


# This requires accurately measuring frequency component amplitudes, so use a
# flat-top window (https://holometer.fnal.gov/GH_FFT.pdf)
flattops = {
    'dantona3': [0.2811, 0.5209, 0.1980],
    'dantona5': [0.21557895, 0.41663158, 0.277263158, 0.083578947,
                 0.006947368],
    'SFT3F': [0.26526, 0.5, 0.23474],
    'SFT4F': [0.21706, 0.42103, 0.28294, 0.07897],
    'SFT5F': [0.1881, 0.36923, 0.28702, 0.13077, 0.02488],
    'SFT3M': [0.28235, 0.52105, 0.19659],
    'SFT4M': [0.241906, 0.460841, 0.255381, 0.041872],
    'SFT5M': [0.209671, 0.407331, 0.281225, 0.092669, 0.0091036],
    'FTSRS': [1.0, 1.93, 1.29, 0.388, 0.028],
    'FTNI': [0.2810639, 0.5208972, 0.1980399],
    'FTHP': [1.0, 1.912510941, 1.079173272, 0.1832630879],
    'HFT70': [1, 1.90796, 1.07349, 0.18199],
    'HFT95': [1, 1.9383379, 1.3045202, 0.4028270, 0.0350665],
    'HFT90D': [1, 1.942604, 1.340318, 0.440811, 0.043097],
    'HFT116D': [1, 1.9575375, 1.4780705, 0.6367431, 0.1228389, 0.0066288],
    'HFT144D': [1, 1.96760033, 1.57983607, 0.81123644, 0.22583558, 0.02773848,
                0.00090360],
    'HFT169D': [1, 1.97441842, 1.65409888, 0.95788186, 0.33673420, 0.06364621,
                0.00521942, 0.00010599],
    'HFT196D': [1, 1.979280420, 1.710288951, 1.081629853, 0.448734314,
                0.112376628, 0.015122992, 0.000871252, 0.000011896],
    'HFT223D': [1, 1.98298997309, 1.75556083063, 1.19037717712, 0.56155440797,
                0.17296769663, 0.03233247087, 0.00324954578, 0.00013801040,
                0.00000132725],
    'HFT248D': [1, 1.985844164102, 1.791176438506, 1.282075284005,
                0.667777530266, 0.240160796576, 0.056656381764, 0.008134974479,
                0.000624544650, 0.000019808998, 0.000000132974],
    }


def THDN(signal, fs, weight=None):
    """Measure the THD+N for a signal and print the results

    Prints the estimated fundamental frequency and the measured THD+N.  This is
    calculated from the ratio of the entire signal before and after
    notch-filtering.

    This notch-filters by nulling out the frequency coefficients ±10% of the
    fundamental

    TODO: Make R vs F reference a parameter (currently is R)
    TODO: Or report all of the above in a dictionary?

    """
    # Get rid of DC and window the signal
    signal = np.asarray(signal) + 0.0  # Float-like array
    # TODO: Do this in the frequency domain, and take any skirts with it?
    signal -= mean(signal)

    window = general_cosine(len(signal), flattops['HFT248D'])
    windowed = signal * window
    del signal

    # Zero pad to nearest power of two
    new_len = next_fast_len(len(windowed))
    windowed = concatenate((windowed, zeros(new_len - len(windowed))))

    # Measure the total signal before filtering but after windowing
    total_rms = rms_flat(windowed)

    # Find the peak of the frequency spectrum (fundamental frequency)
    f = rfft(windowed)
    i = argmax(abs(f))
    true_i = parabolic(log(abs(f)), i)[0]
    frequency = fs * (true_i / len(windowed))

    # Filter out fundamental by throwing away values ±10%
    lowermin = int(true_i * 0.9)
    uppermin = int(true_i * 1.1)
    f[lowermin: uppermin] = 0
    # TODO: Zeroing FFT bins is bad

    # Transform noise back into the time domain and measure it
    noise = irfft(f)
    # TODO: RMS and A-weighting in frequency domain?  Parseval?

    if weight is None:
        pass
    elif weight == 'A':
        # Apply A-weighting to residual noise (Not normally used for
        # distortion, but used to measure dynamic range with -60 dBFS signal,
        # for instance)
        noise = A_weight(noise, fs)
        # TODO: filtfilt? tail end of filter?
    else:
        raise ValueError('Weighting not understood')
    print(rms_flat(noise) / total_rms)
    
    # TODO: Return a dict or list of frequency, THD+N?
    return rms_flat(noise) / total_rms
    print(rms_flat(noise) / total_rms)


def THD(signal, fs):
    """Measure the THD for a signal

    This function is not yet trustworthy.

    Returns the estimated fundamental frequency and the measured THD,
    calculated by finding peaks in the spectrum.

    TODO: Make weighting a parameter
    TODO: Make R vs F reference a parameter (F as default??)

    """
    # Get rid of DC and window the signal
    signal = np.asarray(signal) + 0.0  # Float-like array
    # TODO: Do this in the frequency domain, and take any skirts with it?
    signal -= mean(signal)

    window = general_cosine(len(signal), flattops['HFT248D'])
    windowed = signal * window
    del signal

    # Find the peak of the frequency spectrum (fundamental frequency)
    f = rfft(windowed)
    i = argmax(abs(f))
    true_i = parabolic(log(abs(f)), i)[0]
    #print('Frequency: %f Hz' % (fs * (true_i / len(windowed))))

    print('fundamental amplitude: %.3f' % abs(f[i]))

    # Find the values for the first 15 harmonics.  Includes harmonic peaks
    # only, by definition
    # TODO: Should peak-find near each one, not just assume that fundamental
    # was perfectly estimated.
    # Instead of limited to 15, figure out how many fit based on f0 and
    # sampling rate and report this "4 harmonics" and list the strength of each
    print("Harmonic Peaks:")
    for x in range(2, 15):
        print('%.3f' % abs(f[i * x]), end=' ')

    THD = sum([abs(f[i*x]) for x in range(2, 15)]) / abs(f[i])
    #print(THD)
    print('\n\nTHD: %f%%' % (THD * 100))
    return

analyze("tseries55hz.wav")
analyze("lut55hz.wav")
analyze("sine55hz11.wav")





dBFS values are relative to a full-scale square wave
-----------------
Using sound file backend 'scipy.io.wavfile'
Properties for "tseries55hz.wav"
int16
Channels:	1
Sampling rate:	44100 Hz
Samples:	44800
Length: 	1.0158730158730158 seconds
-----------------
DC offset:	-0.000066 (-0.007%)
Crest factor:	1.415 (3.014 dB)
Peak level:	1.000 (0.000 dBFS)
RMS level:	0.707 (-3.013 dBFS)
RMS A-weighted:	0.027 (-31.525 dBFS(A), -28.511 dB)
RMS 468-weighted:	0.040 (-28.052 dBFS(468), -25.039 dB)
-----------------
Frequency:55.125000
fundamental amplitude: 22391.747
Harmonic Peaks:
21.365 11.495 7.851 5.950 4.791 4.007 3.455 3.026 2.703 2.431 2.210 2.043 1.900 

THD: 0.327031%



dBFS values are relative to a full-scale square wave
-----------------
Using sound file backend 'scipy.io.wavfile'
Properties for "lut55hz.wav"
int16
Channels:	1
Sampling rate:	44100 Hz
Samples:	44800
Length: 	1.0158730158730158 seconds
-----------------
DC offset:	-0.000015 (-0.002%)
Crest factor:	1.416 (3.023 dB)
Pe